<a href="https://colab.research.google.com/github/dhamu2908/DeepLearningAssignment2/blob/main/DeepLearningAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Deep Learning Assignment 2
CS24M027 Dhamodharan Muthu Muniyandi
IIT MADRAS
"""

'\nDeep Learning Assignment 2\nCS24M027 Dhamodharan Muthu Muniyandi\nIIT MADRAS\n'

In [ ]:
import os

#Data Accessing check

print("Exploring contents of /kaggle/input/dldata:")
dataset_path = "/kaggle/input/dldata/inaturalist_12K"
train_path = os.path.join(dataset_path, "train")
val_path = os.path.join(dataset_path, "val")

# Verifying the existence of directories
is_train_available = os.path.isdir(train_path)
is_val_available = os.path.isdir(val_path)


print(f"Train directory found: {is_train_available}")
print(f"Validation directory found: {is_val_available}")

if is_train_available:
    print("Sample items in training folder:", os.listdir(train_path)[:5])

if is_val_available:
    print("Sample items in validation folder:", os.listdir(val_path)[:5])

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import time
import wandb

In [ ]:
# Initialize WandB
wandb.login(key = "" )

In [ ]:
# Define training configuration parameters
config = {
    "num_epochs": 10,
    "lr": 1e-4,
    "batch_sz": 32,
    "backbone": "resnet50",
    "unfreeze_layers": 1
}

IMG_DIM = 224
NUM_CLASSES = 10

# Function to divide dataset into training and validation sets while preserving class balance
def stratified_split(dataset, train_frac):
    train_ids = []
    val_ids = []

    # Index ranges for each class (based on specific dataset arrangement)
    class_boundaries = [
        (0, 999), (1000, 1999), (2000, 2999), (3000, 3999), (4000, 4998),
        (4999, 5998), (5999, 6998), (6999, 7998), (7999, 8998), (8999, 9998)
    ]

    for lower, upper in class_boundaries:
        indices = list(range(lower, upper + 1))
        split_point = int(len(indices) * train_frac)
        train_ids.extend(indices[:split_point])
        val_ids.extend(indices[split_point:])

    training_data = Subset(dataset, train_ids)
    validation_data = Subset(dataset, val_ids)

    return training_data, validation_data


In [ ]:
# Responsible for creating data loaders and returning dataset statistics
def build_data_pipeline(config):
    size = (IMAGE_DIMENSION, IMAGE_DIMENSION)

    def get_transform():
        return transforms.Compose([
            transforms.Resize(size),
            transforms.ToTensor()
        ])

    # Define paths for training and testing datasets
    path_train = "/kaggle/input/nature1/inaturalist_12K/train"
    path_test = "/kaggle/input/nature1/inaturalist_12K/val"

    # Load raw datasets
    complete_train_dataset = ImageFolder(root=path_train, transform=get_transform())
    test_dataset = ImageFolder(root=path_test, transform=get_transform())

    # Split training set into train and validation sets
    splitter = DatasetSplitter(complete_train_dataset, 0.8)
    train_subset, val_subset = splitter.perform_split()

    # Create data loaders
    bs = config["batch_size"]
    loader_train = DataLoader(train_subset, batch_size=bs, shuffle=True)
    loader_val = DataLoader(val_subset, batch_size=bs, shuffle=True)
    loader_test = DataLoader(test_dataset, batch_size=bs, shuffle=True)

    # Return everything in a structured dictionary
    return {
        "train_size": len(train_subset),
        "val_size": len(val_subset),
        "test_size": len(test_dataset),
        "train_loader": loader_train,
        "val_loader": loader_val,
        "test_loader": loader_test
    }
